In [ ]:
!nvidia-smi

In [ ]:
!pip install transformers==4.18.0
!pip install -U datasets

In [ ]:
pip install --upgrade --force-reinstall pyarrow

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import json # to read json
from datasets import load_dataset,Dataset,DatasetDict
import warnings # to ignore warnings
warnings.filterwarnings('ignore')

In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
def squad_json_to_dataframe_train(input_file_path, record_path = ['data','paragraphs','qas','answers'],
                           verbose = 1):
    """
    input_file_path: path to the squad json file.
    record_path: path to deepest level in json file default value is
    ['data','paragraphs','qas','answers']
    verbose: 0 to suppress it default is 1
    """
    if verbose:
        print("Reading the json file")    
    file = json.loads(open(input_file_path).read())
    if verbose:
        print("processing...")
    # parsing different level's in the json file
    js = pd.io.json.json_normalize(file , record_path )
    m = pd.io.json.json_normalize(file, record_path[:-1] )
    r = pd.io.json.json_normalize(file,record_path[:-2])
    
    #combining it into single dataframe
    idx = np.repeat(r['context'].values, r.qas.str.len())
    ndx  = np.repeat(m['id'].values,m['answers'].str.len())
    m['context'] = idx
    js['q_idx'] = ndx
    main = pd.concat([ m[['id','question','context']].set_index('id'),js.set_index('q_idx')],1,sort=False).reset_index()
    main['c_id'] = main['context'].factorize()[0]
    if verbose:
        print("shape of the dataframe is {}".format(main.shape))
        print("Done")
    return main

__notebook_source__.ipynb


In [11]:
# training data
language = 'ar'
input_file_path = f'../input/squadenar/squad.translate.train.en-{language}.json'
record_path = ['data','paragraphs','qas','answers']
train = squad_json_to_dataframe_train(input_file_path=input_file_path,record_path=record_path,verbose=0)

In [12]:
train.head()

,index,question,context,answer_start,text,c_id
0,5706071252bb8914006897cd,أين تزود المهاجرين بالوقود؟,بالنسبة لبعض أنواع الخواضون ، يعتمد نجاح الهجر...,86,نقاط التوقف على طول طريق الترحيل,0
1,56f82171a6d7ea1400e173e9,من المشهور الذي عبر جبال الألب مع قطيع من الفيلة؟,اكتشف رجل محنط ، يبلغ عمره 5000 عام ، على نهر ...,165,هانيبال,1
2,5726c1e7708984140094d0a2,متى غادر الرومان أوترخت,من منتصف القرن الثالث غزت القبائل الجرمانية با...,71,حوالي 275 لم يعد بإمكان الرومان الحفاظ على الح...,2
3,570e6b020b85d914000d7eb9,ما هي النظرية المسماة التعامل مع نقل السنسكريت...,من أجل شرح الميزات الشائعة التي تتقاسمها اللغة...,118,نظرية هجرة الهند الآرية,3
4,5733cf6a4776f419006612a4,ماذا تمتلك المعاهدة التي يفتقر إليها الآخرون؟,في عام 1998 ، صرح كورت جوناسون وكارين بيورنسون...,262,مصداقية قانونية دولية,4


In [13]:
def get_answers(x):
    start = x[0]
    text = x[1]
    return {
        'answer_start': [start],
        'text': [text]
    }

train['answers'] = train[['answer_start', 'text']].apply(get_answers, axis=1)
pd.set_option('display.max_colwidth',None)
train.head(1)

,index,question,context,answer_start,text,c_id,answers
0,5706071252bb8914006897cd,أين تزود المهاجرين بالوقود؟,بالنسبة لبعض أنواع الخواضون ، يعتمد نجاح الهجرة على توفر بعض موارد الغذاء الرئيسية في نقاط التوقف على طول طريق الترحيل. هذا يعطي المهاجرين فرصة للتزود بالوقود من أجل المحطة التالية من الرحلة. بعض الأمثلة على مواقع التوقف المهمة هي Bay of Fundy و Delaware Bay.,86,نقاط التوقف على طول طريق الترحيل,0,"{'answer_start': [86], 'text': ['نقاط التوقف على طول طريق الترحيل']}"


In [14]:
# Validation data
language = 'ar'
input_file_path = f'../input/squadenar/squad.translate.dev.en-{language}.json'
record_path = ['data','paragraphs','qas','answers']
dev = squad_json_to_dataframe_train(input_file_path=input_file_path,record_path=record_path,verbose=0)
dev['answers'] = dev[['answer_start', 'text']].apply(get_answers, axis=1)
pd.set_option('display.max_colwidth',None)
dev.head(1)

,index,question,context,answer_start,text,c_id,answers
0,5729da0faf94a219006aa678,ما نوع الأجور الناتجة عن الوظائف التي يكون فيها العرض منخفضًا ولكن الطلب مرتفعًا؟,إن الوظيفة التي يوجد فيها عدد كبير من العمال الراغبين في العمل لفترة طويلة من الوقت (ارتفاع العرض) يتنافسون على وظيفة تتطلبها قلة (انخفاض الطلب) ستؤدي إلى انخفاض الأجر عن هذه الوظيفة. وذلك لأن المنافسة بين العمال تدفع الأجور. مثال على ذلك سيكون وظائف مثل غسل الصحون أو خدمة العملاء. تميل المنافسة بين العمال إلى خفض الأجور بسبب الطبيعة المستهلكة للعامل فيما يتعلق بوظيفته. إن الوظيفة التي يوجد فيها عدد قليل من العمال القادرين أو الراغبين (نقص العرض) ، ولكن هناك حاجة كبيرة للوظائف (ارتفاع الطلب) ، ستؤدي إلى ارتفاع أجر لهذه الوظيفة. وذلك لأن التنافس بين أرباب العمل على الموظفين سيؤدي إلى رفع الأجور. تشمل الأمثلة على ذلك الوظائف التي تتطلب مهارات متطورة للغاية ، أو قدرات نادرة ، أو درجة عالية من المخاطرة. تميل المنافسة بين أرباب العمل إلى رفع الأجور بسبب طبيعة الوظيفة ، نظرًا لوجود نقص نسبي في عدد العمال في هذا المنصب المحدد. قد تحد المنظمات المهنية والعمال من عرض العمال مما يؤدي إلى زيادة الطلب وزيادة الدخول للأعضاء. قد يحصل الأعضاء أيضًا على أجور أعلى من خلال المفاوضة الجماعية أو التأثير السياسي أو الفساد.,509,ارتفاع,0,"{'answer_start': [509], 'text': ['ارتفاع']}"


In [15]:
dev.head()

,index,question,context,answer_start,text,c_id,answers
0,5729da0faf94a219006aa678,ما نوع الأجور الناتجة عن الوظائف التي يكون فيها العرض منخفضًا ولكن الطلب مرتفعًا؟,إن الوظيفة التي يوجد فيها عدد كبير من العمال الراغبين في العمل لفترة طويلة من الوقت (ارتفاع العرض) يتنافسون على وظيفة تتطلبها قلة (انخفاض الطلب) ستؤدي إلى انخفاض الأجر عن هذه الوظيفة. وذلك لأن المنافسة بين العمال تدفع الأجور. مثال على ذلك سيكون وظائف مثل غسل الصحون أو خدمة العملاء. تميل المنافسة بين العمال إلى خفض الأجور بسبب الطبيعة المستهلكة للعامل فيما يتعلق بوظيفته. إن الوظيفة التي يوجد فيها عدد قليل من العمال القادرين أو الراغبين (نقص العرض) ، ولكن هناك حاجة كبيرة للوظائف (ارتفاع الطلب) ، ستؤدي إلى ارتفاع أجر لهذه الوظيفة. وذلك لأن التنافس بين أرباب العمل على الموظفين سيؤدي إلى رفع الأجور. تشمل الأمثلة على ذلك الوظائف التي تتطلب مهارات متطورة للغاية ، أو قدرات نادرة ، أو درجة عالية من المخاطرة. تميل المنافسة بين أرباب العمل إلى رفع الأجور بسبب طبيعة الوظيفة ، نظرًا لوجود نقص نسبي في عدد العمال في هذا المنصب المحدد. قد تحد المنظمات المهنية والعمال من عرض العمال مما يؤدي إلى زيادة الطلب وزيادة الدخول للأعضاء. قد يحصل الأعضاء أيضًا على أجور أعلى من خلال المفاوضة الجماعية أو التأثير السياسي أو الفساد.,509,ارتفاع,0,"{'answer_start': [509], 'text': ['ارتفاع']}"
1,5729da0faf94a219006aa678,ما نوع الأجور الناتجة عن الوظائف التي يكون فيها العرض منخفضًا ولكن الطلب مرتفعًا؟,إن الوظيفة التي يوجد فيها عدد كبير من العمال الراغبين في العمل لفترة طويلة من الوقت (ارتفاع العرض) يتنافسون على وظيفة تتطلبها قلة (انخفاض الطلب) ستؤدي إلى انخفاض الأجر عن هذه الوظيفة. وذلك لأن المنافسة بين العمال تدفع الأجور. مثال على ذلك سيكون وظائف مثل غسل الصحون أو خدمة العملاء. تميل المنافسة بين العمال إلى خفض الأجور بسبب الطبيعة المستهلكة للعامل فيما يتعلق بوظيفته. إن الوظيفة التي يوجد فيها عدد قليل من العمال القادرين أو الراغبين (نقص العرض) ، ولكن هناك حاجة كبيرة للوظائف (ارتفاع الطلب) ، ستؤدي إلى ارتفاع أجر لهذه الوظيفة. وذلك لأن التنافس بين أرباب العمل على الموظفين سيؤدي إلى رفع الأجور. تشمل الأمثلة على ذلك الوظائف التي تتطلب مهارات متطورة للغاية ، أو قدرات نادرة ، أو درجة عالية من المخاطرة. تميل المنافسة بين أرباب العمل إلى رفع الأجور بسبب طبيعة الوظيفة ، نظرًا لوجود نقص نسبي في عدد العمال في هذا المنصب المحدد. قد تحد المنظمات المهنية والعمال من عرض العمال مما يؤدي إلى زيادة الطلب وزيادة الدخول للأعضاء. قد يحصل الأعضاء أيضًا على أجور أعلى من خلال المفاوضة الجماعية أو التأثير السياسي أو الفساد.,509,ارتفاع,0,"{'answer_start': [509], 'text': ['ارتفاع']}"
2,56e761d037bdd419002c3f17,كيف يصنف الطلاب المعلمين الذين يشعرون أنهم متحمسون؟,نظرًا لأنه يمكن للمعلمين التأثير على كيفية فهم الطلاب لمواد الدورة التدريبية ، فقد وجد أن المدرسين الذين أبدوا حماسة تجاه المواد الدراسية والطلاب يمكن أن يؤثروا في تجربة تعليمية إيجابية تجاه مواد الدورة التدريبية. فيما يتعلق بتقييمات المدرس / الدورة التدريبية ، وجد أن المعلمين الذين لديهم ميل إيجابي نحو محتوى الدورة التدريبية يميلون إلى نقل شغفهم إلى الطلاب المستقبليين. هؤلاء المعلمون لا يقومون بالتدريس عن ظهر قلب ولكنهم يحاولون العثور على تنشيط جديد لمواد الدورة على أساس يومي. واحدة من الصعوبات في هذا النهج هو أن المعلمين ربما قاموا بتغطية المناهج الدراسية مرارًا وتكرارًا حتى يبدأوا في الشعور بالملل من الموضوع الذي يؤدي بدوره إلى غضب الطلاب أيضًا. يميل الطلاب الذين لديهم مدرسون متحمسون إلى تقييمهم أعلى من المعلمين الذين لم يبدوا الكثير من الحماس لمواد الدورة التدريبية.,708,أعلى,1,"{'answer_start': [708], 'text': ['أعلى']}"
3,56e761d037bdd419002c3f17,كيف يصنف الطلاب المعلمين الذين يشعرون أنهم متحمسون؟,نظرًا لأنه يمكن للمعلمين التأثير على كيفية فهم الطلاب لمواد الدورة التدريبية ، فقد وجد أن المدرسين الذين أبدوا حماسة تجاه المواد الدراسية والطلاب يمكن أن يؤثروا في تجربة تعليمية إيجابية تجاه مواد الدورة التدريبية. فيما يتعلق بتقييمات المدرس / الدورة التدريبية ، وجد أن المعلمين الذين لديهم ميل إيجابي نحو محتوى الدورة التدريبية يميلون إلى نقل شغفهم إلى الطلاب المستقبليين. هؤلاء المعلمون لا يقومون بالتدريس عن ظهر قلب ولكنهم يحاولون العثور على تنشيط جديد لمواد الدورة على أساس يومي. واحدة من الصعوبات في هذا النهج هو أن المعلمين ربما قاموا بتغطية المناهج الدراسية مرارًا وتكرارًا

In [43]:
tds = Dataset.from_pandas(train)
vds = Dataset.from_pandas(dev)


squad = DatasetDict()

squad['train'] = tds
squad['validation'] = vds

In [44]:
squad

DatasetDict({
    train: Dataset({
        features: ['index', 'question', 'context', 'answer_start', 'text', 'c_id', 'answers'],
        num_rows: 86787
    })
    validation: Dataset({
        features: ['index', 'question', 'context', 'answer_start', 'text', 'c_id', 'answers'],
        num_rows: 34448
    })
})

In [45]:
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

pretraining_language = 'ar'
# model = AutoModelForQuestionAnswering.from_pretrained("microsoft/Multilingual-MiniLM-L12-H384")
model = AutoModelForQuestionAnswering.from_pretrained(f"subhasisj/{pretraining_language}-TAPT-MLM-MiniLM")
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

loading configuration file https://huggingface.co/subhasisj/ar-TAPT-MLM-MiniLM/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/f1e964ae98e6ee43ab8ed956aa7f1a59a9d86d02afa1ecfe1b753de417eaff93.e10bd0b72b4099c977ef126c1372e211950889239eb7513aba618c575d204753
Model config BertConfig {
  "_name_or_path": "subhasisj/ar-TAPT-MLM-MiniLM",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 1536,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "tokenizer_class": "XLMRobertaTokenizer",
  "torch_dtype": "float32",
  "transformers_version": "4.18.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 250037
}

load

In [46]:
pad_on_right = tokenizer.padding_side == "right"
max_length = 384 # The maximum length of a feature (question and context)
doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.
def prepare_train_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

def prepare_validation_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["index"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

In [47]:
batch_size = 32

In [48]:
squad = squad.map(prepare_train_features,batched=True,remove_columns=squad["train"].column_names)

  0%|          | 0/87 [00:00<?, ?ba/s]

  0%|          | 0/35 [00:00<?, ?ba/s]

In [49]:
from transformers import Trainer, TrainingArguments
args = TrainingArguments(
    f"./{pretraining_language}-finetuned-squad-qa-minilmv2-{batch_size}",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate = 3e-5,
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size = batch_size,
    warmup_ratio = 0.1,
    gradient_accumulation_steps = 8,
    num_train_epochs = 5,
    fp16=True,
    weight_decay = 0.01,
#     push_to_hub=True,
    save_total_limit=1,
    load_best_model_at_end=True
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [50]:
from transformers import default_data_collator

data_collator = default_data_collator

In [51]:
trainer = Trainer(
    model,
    args,
    train_dataset=squad["train"],
    eval_dataset=squad["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

Using amp half precision backend


In [52]:
trainer.train()

***** Running training *****
  Num examples = 90100
  Num Epochs = 5
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 256
  Gradient Accumulation steps = 8
  Total optimization steps = 1760
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Epoch,Training Loss,Validation Loss
1,No log,2.449250
2,3.236200,1.805774
3,1.713900,1.574798
4,1.713900,1.496392
5,1.341700,1.477190


***** Running Evaluation *****
  Num examples = 36077
  Batch size = 32
Saving model checkpoint to ./ar-finetuned-squad-qa-minilmv2-32/checkpoint-352
Configuration saved in ./ar-finetuned-squad-qa-minilmv2-32/checkpoint-352/config.json
Model weights saved in ./ar-finetuned-squad-qa-minilmv2-32/checkpoint-352/pytorch_model.bin
tokenizer config file saved in ./ar-finetuned-squad-qa-minilmv2-32/checkpoint-352/tokenizer_config.json
Special tokens file saved in ./ar-finetuned-squad-qa-minilmv2-32/checkpoint-352/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 36077
  Batch size = 32
Saving model checkpoint to ./ar-finetuned-squad-qa-minilmv2-32/checkpoint-704
Configuration saved in ./ar-finetuned-squad-qa-minilmv2-32/checkpoint-704/config.json
Model weights saved in ./ar-finetuned-squad-qa-minilmv2-32/checkpoint-704/pytorch_model.bin
tokenizer config file saved in ./ar-finetuned-squad-qa-minilmv2-32/checkpoint-704/tokenizer_config.json
Special tokens file saved in ./a

TrainOutput(global_step=1760, training_loss=1.9717293999411842, metrics={'train_runtime': 9396.3426, 'train_samples_per_second': 47.944, 'train_steps_per_second': 0.187, 'total_flos': 2.2103297862912e+16, 'train_loss': 1.9717293999411842, 'epoch': 5.0})

In [53]:
trainer.push_to_hub("Training Completed.")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


OSError: Looks like you do not have git-lfs installed, please install. You can install from https://git-lfs.github.com/. Then run `git lfs install` (you only have to do this once).

In [ ]:
validation_features = squad["validation"]

In [ ]:
# validation_features = squad_valid.map(
#     prepare_validation_features,
#     batched=True,
#     remove_columns=squad_valid.column_names
# )

In [55]:
!tar -zcvf outputname.tar.gz /kaggle/working/ar-finetuned-squad-qa-minilmv2-32/checkpoint-1760

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
tar: Removing leading `/' from member names
/kaggle/working/ar-finetuned-squad-qa-minilmv2-32/checkpoint-1760/
/kaggle/working/ar-finetuned-squad-qa-minilmv2-32/checkpoint-1760/tokenizer_config.json
/kaggle/working/ar-finetuned-squad-qa-minilmv2-32/checkpoint-1760/config.json
/kaggle/working/ar-finetuned-squad-qa-minilmv2-32/checkpoint-1760/trainer_state.json
/kaggle/working/ar-finetuned-squad-qa-minilmv2-32/checkpoint-1760/rng_state.pth
/kaggle/working/ar-finetuned-squad-qa-minilmv2-32/checkpoint-1760/special_tokens_map.json
/kaggle/working/ar-finetuned-squad-qa-minilmv2-32/checkpoint-1760/scheduler.pt
/kaggle/working/ar-finetuned-squad-qa-minilmv2-32/checkpoint-1760

In [57]:
import os
os.chdir(r'/kaggle/working')
from IPython.display import FileLink

FileLink(r'./outputname.tar.gz')

/kaggle/working/outputname.tar.gz

In [54]:
import os
os.chdir(r'/kaggle/working')
from IPython.display import FileLink
FileLink(f"./{pretraining_language}-finetuned-squad-qa-minilmv2-{batch_size}/pytorch_model.bin"')

SyntaxError: EOL while scanning string literal (973696571.py, line 4)

In [ ]:
FileLink(f"./{pretraining_language}-finetuned-squad-qa-minilmv2-{batch_size}/config.json"')

In [ ]:
FileLink(f"./{pretraining_language}-finetuned-squad-qa-minilmv2-{batch_size}/tokenizer_config.json"')

In [ ]:
FileLink(f"./{pretraining_language}-finetuned-squad-qa-minilmv2-{batch_size}/special_tokens_map.json"')

In [ ]:
raw_predictions = trainer.predict(validation_features)

In [ ]:
validation_features.set_format(type=validation_features.format["type"], columns=list(validation_features.features.keys()))

In [ ]:
import collections

examples = squad_valid
features = validation_features

example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
features_per_example = collections.defaultdict(list)
for i, feature in enumerate(features):
    features_per_example[example_id_to_index[feature["example_id"]]].append(i)

In [ ]:
from tqdm.auto import tqdm

def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
    all_start_logits, all_end_logits = raw_predictions
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.OrderedDict()

    # Logging.
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None # Only used if squad_v2 is True.
        valid_answers = []
        
        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                        or offset_mapping[start_index] == []
                        or offset_mapping[end_index] == []
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0}
        
        # Let's pick our final answer: the best one or the null answer (only for squad_v2)
        # if not squad_v2:
            # predictions[example["id"]] = best_answer["text"]
        # else:
        answer = best_answer["text"] if best_answer["score"] > min_null_score else ""
        predictions[example["id"]] = answer

    return predictions

In [ ]:
final_predictions = postprocess_qa_predictions(squad_valid, validation_features, raw_predictions.predictions)


In [ ]:
from datasets import load_metric
metric = load_metric("squad")

In [ ]:
formatted_predictions = [{"id": k, "prediction_text": v} for k, v in final_predictions.items()]
references = [{"id": ex["id"], "answers": ex["answers"]} for ex in squad_valid]
metric.compute(predictions=formatted_predictions, references=references)